In [1]:
import cv2 as cv
import matplotlib.pyplot as plt
import json
import math
import numpy as np

In [2]:
from common_lib import DataManager
from common_lib import AnnotationManager

In [3]:
'''
显示图片
输入：图片矩阵，画布大小：元组（宽，高）
'''
def showImg(img,figsize=(12,12),cmap = None):
    plt.figure(figsize=figsize)
    plt.axis('off')
    plt.imshow(img,cmap = cmap)
    plt.show()

In [4]:
'''
在img中画出对应点
输出：box是点的坐标数组,img是对应图片
输出：标上点的图片
'''
def showPoint(img,box):
    point_size = 20
    point_color = (0, 0, 255) # BGR
    thickness = 8 # 可以为 0 、4、8
    for item in box:
#     print((int(item[0]),int(item[1])))
        cv.circle(img, (int(item[0]),int(item[1])), point_size, point_color, thickness)
#     plt.imshow(img)
    return img

In [5]:
'''
获取圆心坐标
输入：所有的图片标注数据（包括图片id，同心圆，框框点坐标）：json文件，图片的id ：字符串
输出：查找图片的圆心坐标：形式[x,y]
'''
def getCicleCenter(imgtotalData,img_id):
    centerX = 0
    centerY = 0
    circles = []
    for item in imgtotalData['record']:
        if(item['info']['uuid']==img_id):
            circles = item['info']['circles']
            break
    for item in circles:
        centerX += item[0]
        centerY += item[1]
    centerX /= len(circles)
    centerY /= len(circles)
    result = [centerX,centerY]
    return result

####  通过json中的info信息获取圆心坐标

In [6]:
'''
通过json中的info信息获取圆心坐标
输入：rec表示一张图片的json信息
输出：查找图片的圆心坐标：形式[x,y]
'''
def getCicleCenterByInfo(rec):
    centerX = 0
    centerY = 0
    circles = rec['info']['circles']
    for item in circles:
        centerX += item[0]
        centerY += item[1]
    centerX /= len(circles)
    centerY /= len(circles)
    result = [centerX,centerY]
    return result

In [7]:
'''
获取圆信息
输出：所有的图片标注数据（包括图片id，同心圆，框框点坐标）：json文件，图片的id ：字符串
输出：查找图片的圆信息
'''
def getCicleInfo(imgtotalData,img_id):
    circles = []
    for item in imgtotalData['record']:
        if(item['info']['uuid']==img_id):
            circles = item['info']['circles']
            break
    return circles

In [8]:
'''
得到多边形最大最小值信息
输入：多边形的点的坐标list
输出：相关最大最小值信息
'''
def getInfo(box): 
    Xmin = box[0][0]
    Xmax = box[0][0]
    Ymin = box[0][1]
    Ymax = box[0][1]
    for item in box:
        if(item[0]>Xmax):
            Xmax = item[0]
        if(item[0]<Xmin):
            Xmin = item[0] 
        if(item[1]>Ymax):
            Ymax = item[1]
        if(item[1]<Ymin):
            Ymin = item[1]
    result = [Xmin,Ymin,Xmax,Ymax]
    return result


In [9]:
'''
得到字符的中心点
输入：多边形的点的坐标：list形式[[x,y],...]
输出：中心点坐标
'''
def getCharacterCenter(box):
    Info = getInfo(box)
    result = [(Info[0]+Info[2])/2,(Info[1]+Info[3])/2,]
    result = [int(i) for i in result]
    return result

#### 通过json的getxyx获取的信息得到中心点坐标

In [10]:
'''
得到字符的中心点
输入：相关最大最小值信息
输出：中心点坐标
'''
def getCharacterCenterByInfo(Info):
#     Info = getInfo(box)
    result = [(Info[0]+Info[2])/2,(Info[1]+Info[3])/2,]
    result = [int(i) for i in result]
    return result

In [11]:
'''
得到剪裁的图片
输入：图像，多边形的点的坐标：list形式[[x,y],...]
输出：剪裁好的图片
'''
def getCropped(img,box):
    result = getInfo(box)
    result = [int(i) for i in result]
    cropped = img[result[1]:result[3],result[0]:result[2]]
#     showImg(cropped,(4,4))
    return cropped

#### 通过json的getxyx获取的信息得到剪裁图片

In [12]:
'''
通过json的getxyx获取的信息得到剪裁图片
输入：图像，多边形的点的坐标：list形式[[x,y],...]
输出：剪裁好的图片
'''
def getCroppedByInfo(img,Info):
    result = [int(i) for i in Info]
    cropped = img[result[1]:result[3],result[0]:result[2]]
#     showImg(cropped,(4,4))
    return cropped

In [13]:
'''
根据图片中心点，和指定的正方形边长得到剪裁的图片
输入：图像，多边形的中心点的坐标：list形式[x,y]，指定
输出：剪裁好的图片
'''
def getCroppedBycenter(img,CharacterCenter,Rlength):
    CharacterCenter = [int(i) for i in CharacterCenter]
    span = int(Rlength/2)
    cropped = img[CharacterCenter[1]-span:CharacterCenter[1]+span,CharacterCenter[0]-span:CharacterCenter[0]+span]
    return cropped

In [14]:
'''
计算绕中心点旋转角度 顺时针为负 逆时针为正
输入：中心点的[x,y] 圆心坐标[x,y]
输出：旋转度数
'''
def calcAngle(point,center):
    angle = 0
    xdis = center[0]-point[0]
    ydis = center[1]-point[1]
    if(xdis!=0 and ydis!=0):
        tempAngle = math.atan(abs(xdis)/abs(ydis))*180/math.pi
        if(xdis>0 and ydis>0):
            angle = -(tempAngle) #右旋
        if(xdis<0 and ydis>0):
            angle = tempAngle
        if(xdis>0 and ydis<0):
            angle = tempAngle-180
        if(xdis<0 and ydis<0):
            angle = 180-tempAngle
    if(xdis==0 and ydis<0):
        angle = 180
    if(ydis==0 and xdis>0):
        angle = -90
    if(ydis==0 and xdis<0):
        angle = 90
    return angle

In [27]:
#角度为负，顺时针；角度为正，逆时针
'''
得到旋转后图片
输入：剪裁图片和旋转角度
输出：将剪裁图片旋转angle的图像
'''
def rotateImg(cropped,angle=0,pad=0,borderValue = 0):
    rows,cols = cropped.shape[:2]
    # cols-1 and rows-1 are the coordinate limits.
    M = cv.getRotationMatrix2D(((cols-1)/2.0,(rows-1)/2.0),angle,1)
    dst = cv.warpAffine(cropped,M,(cols+pad,rows+pad),borderValue=(borderValue,borderValue,borderValue))
    return dst

In [16]:
'''
通过圆心，字符框得到切割矫正后的图片
输入：圆心坐标：[x,y],多边形框的顶点位置：[[x,y],...],旋转后填充的长宽像素值
输出：剪裁矫正后的字符图像
'''
def getPattern(img,circleCenter,box,pad=0):
    characterCenter = getCharacterCenter(box)
    angle = calcAngle(characterCenter,circleCenter) #图片的圆心坐标 旋转角度
    cropped = getCropped(img,box)                       #得到剪裁的字符图片
    result = rotateImg(cropped,angle,pad)
    return result

In [37]:
'''
通过圆心，字符中心点得到切割矫正后的图片
输入：圆心坐标：[x,y],多边形框的顶点位置：[[x,y],...],旋转后填充的长宽像素值
输出：剪裁矫正后的字符图像
'''
def getPatternByCenter(img,circleCenter,characterCenter,Rlength,pad=0,boderValue = 0):
    angle = calcAngle(characterCenter,circleCenter) #图片的圆心坐标 旋转角度
    cropped = getCroppedBycenter(img,characterCenter,Rlength)    #得到剪裁的字符图片
    result = rotateImg(cropped,angle,pad,boderValue)
    return result

#### 通过json得到的info来替换box的功能 利用外接矩形

In [33]:
'''
通过圆心，字符框得到切割矫正后的图片
输入：圆心坐标：[x,y],Info多边形框的[Xmin,Ymin,Xmax,Ymax]：[[x,y],...],旋转后填充的长宽像素值
输出：剪裁矫正后的字符图像
'''
def getPatternByInfo(img,circleCenter,Info,pad=0,boderValue = 0):
    characterCenter = getCharacterCenterByInfo(Info)
    angle = calcAngle(characterCenter,circleCenter) #图片的圆心坐标 旋转角度
    cropped = getCroppedByInfo(img,Info)                       #得到剪裁的字符图片
    result = rotateImg(cropped,angle,pad,boderValue)
    return result

## 处理全部

In [41]:
data = DataManager.load('labeled_giant_data.json')
image_data_root = 'F:/mountMl_fan/bearing/'

### 使用中心点指定边长剪裁旋转所有数据

In [54]:
'''
根据json文件剪裁旋转字符图片
输入：data（json文件）,image_data_root:图像存放的根目录，Rlength ：剪裁的正方形边长,
boderValue 表示边缘填充的颜色 默认设置为黑色  cropRlength=100 得到旋转图像后扣取中心部分
输出：img_list表示裁剪并旋转后返回的图片列表
'''
def cropRotate(data,image_data_root,Rlength=200,boderValue=0,cropRlength=100):
    am = AnnotationManager(data.class_dict)
    img_list = []
    Cstart = int((Rlength - cropRlength)/2)
    Cend = int(Cstart+cropRlength)
    for rec in data:
        image_path = image_data_root + rec['info']['image_path']
        img = cv.imread(image_path)
        circleCenter = getCicleCenterByInfo(rec)
        rec_id = rec['info']['uuid']
        for inst in rec['instances']:
            character_id = inst['uuid']
            class_names = am.get_classname(inst)
            if('word' in class_names):
                Info = am.get_xyxy(inst)
                characterCenter = getCharacterCenterByInfo(Info)
                result = getPatternByCenter(img,circleCenter,characterCenter,Rlength,boderValue=boderValue)
                result = result[Cstart:Cend,Cstart:Cend]
                img_list.append(result)
                cv.imwrite("./result/all_rotate_result_big/{}.png".format(rec_id+"_"+character_id),result)
    return img_list


In [56]:
'''
根据json文件剪裁字符图片
输入：data（json文件）,image_data_root:图像存放的根目录，Rlength ：剪裁的正方形边长
输出：img_list表示裁剪后返回的图片列表
'''
def cropImg(data,image_data_root,Rlength):
    am = AnnotationManager(data.class_dict)
    img_list = []
    for rec in data:
        image_path = image_data_root + rec['info']['image_path']
        img = cv.imread(image_path)
        circleCenter = getCicleCenterByInfo(rec)
        rec_id = rec['info']['uuid']
        for inst in rec['instances']:
            character_id = inst['uuid']
            class_names = am.get_classname(inst)
            if('word' in class_names):
                Info = am.get_xyxy(inst)
                characterCenter = getCharacterCenterByInfo(Info)
                result = getCroppedBycenter(img,characterCenter,Rlength)
                img_list.append(result)
                cv.imwrite("./result/all_cropped_result/{}.png".format(rec_id+"_"+character_id),result)
    return img_list

In [57]:
img_list = cropRotate(data,image_data_root,200,255,cropRlength=100)

In [42]:
img_list = cropImg(data,image_data_root,100)

## 读取图像和json信息

In [77]:
img = cv.imread("./1_2.png")
filename = './1_2.json'
with open(filename) as f:
    imgData = json.load(f)
with open('./labeled_giant_data.json') as f:
    imgtotalData = json.load(f)
    
#单个图片
img_id = imgData['uuid']    #图片的id
boxlist = imgData['shapes'] #多边形框点列表

## 通过多边形得到字符图像并保存

In [19]:
i=0
circleCenter = getCicleCenter(imgtotalData,img_id) #得到圆中心点
pad = 5
for item in boxlist: #处理一张图的多边形
    if(item['label']=='word'):   #筛选出图像中的字符元素
        boxItem = item['points'] #得到此元素的box点的位置[[][]] 二维列表 [x,y] 定位图像中的某个点
        result = getPattern(img,circleCenter,boxItem,pad)
        cv.imwrite("./rotate_result/{}.png".format(str(i)+"picture"),result)
        i += 1
#         cropped = getCroped(boxItem) 
#         cv.imwrite("./cropped_result/{}.png".format(str(i)+"picture"),cropped)
#         i += 1

## 通过字符中心点得到字符图像并保存

In [47]:
getCicleInfo(imgtotalData,img_id)

[[1984.839111328125, 1450.1273193359375, 558.5774841308594],
 [1984.755859375, 1450.391845703125, 717.52099609375],
 [1985.1937255859375, 1450.6708984375, 1032.5601196289062],
 [1985.54296875, 1449.3905029296875, 1155.46533203125],
 [1985.563720703125, 1453.12939453125, 1184.8939208984375]]

In [36]:
## 文件保存
i=0
circleCenter = getCicleCenter(imgtotalData,img_id) #得到圆中心点
pad = 0
Rlength = 100
for item in boxlist: #处理一张图的多边形
    if(item['label']=='word'):   #筛选出图像中的字符元素
        boxItem = item['points'] #得到此元素的box点的位置[[][]] 二维列表 [x,y] 定位图像中的某个点
        characterCenter = getCharacterCenter(boxItem)
        result = getPatternByCenter(img,circleCenter,characterCenter,Rlength,pad=pad)
        cv.imwrite("./rotateByCenter_result/{}.png".format(str(i)+"picture"),result)
        i += 1
#         cropped = getCroped(boxItem) 
#         cv.imwrite("./cropped_result/{}.png".format(str(i)+"picture"),cropped)
#         i += 1

In [78]:
## 使用列表保存
i=0
circleCenter = getCicleCenter(imgtotalData,img_id) #得到圆中心点
pad = 0
Rlength = 100
imgList = []
for item in boxlist: #处理一张图的多边形
    if(item['label']=='word'):   #筛选出图像中的字符元素
        boxItem = item['points'] #得到此元素的box点的位置[[][]] 二维列表 [x,y] 定位图像中的某个点
        characterCenter = getCharacterCenter(boxItem)
        result = getPatternByCenter(img,circleCenter,characterCenter,Rlength,pad=pad)
        imgList.append(result)
#         cv.imwrite("./rotateByCenter_result/{}.png".format(str(i)+"picture"),result)
#         i += 1
#         cropped = getCroped(boxItem) 
#         cv.imwrite("./cropped_result/{}.png".format(str(i)+"picture"),cropped)
#         i += 1

## 二值化图像并消除噪音

In [80]:
# img = cv.imread("./rotateByCenter_result/0picture.png",cv.IMREAD_GRAYSCALE) #以灰度图形式读取文件 shape为100*100
maskList = []
i=0
for item in imgList:
    img = cv.cvtColor(item,cv.COLOR_BGR2GRAY)
    npImg = np.array(img)
    npImg[(npImg<80)&(npImg>0)] = 255
    npImg[npImg<255] = 0
    mask = cv.erode(npImg.astype('uint8'), kernel=np.ones((2,2)),iterations=3)
    cv.imwrite("./result/maskFile/{}.png".format(str(i)+"picture"),mask)
    maskList.append(mask)
    i+=1